<a href="https://colab.research.google.com/github/anupojuharshita/Lane-Detection/blob/main/cnn%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -----------------------------
# Complete Code with Noise Reduction and Clear Visualization
# -----------------------------
import zipfile
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2

# -----------------------------
# 1. Unzip Combined Dataset
# -----------------------------
zip_path = "/content/combined_dataset.zip"
extract_path = "/content/combined_dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"✅ Unzipped {zip_path} to {extract_path}")

# -----------------------------
# 2. Custom Dataset for Combined Images
# -----------------------------
class CombinedLaneDataset(Dataset):
    def __init__(self, combined_dir):
        self.combined_dir = combined_dir
        self.frame_names = sorted(os.listdir(combined_dir))

        self.input_transform = transforms.Compose([
            transforms.Resize((128, 128)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

        self.output_transform = transforms.Compose([
            transforms.Resize((128, 128)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.frame_names)

    def __getitem__(self, idx):
        combined_path = os.path.join(self.combined_dir, self.frame_names[idx])
        combined_image = Image.open(combined_path).convert('RGB')

        w, h = combined_image.size
        input_image = combined_image.crop((0, 0, w // 2, h))
        output_image = combined_image.crop((w // 2, 0, w, h)).convert('L')

        input_tensor = self.input_transform(input_image)
        output_tensor = self.output_transform(output_image)

        # Ensure the mask has only 1 channel
        if output_tensor.shape[0] != 1:
            output_tensor = output_tensor[0].unsqueeze(0)

        return input_tensor, output_tensor

# -----------------------------
# 3. Simple CNN Model for Lane Detection
# -----------------------------
class LaneCNN(nn.Module):
    def __init__(self):
        super(LaneCNN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 2, stride=2), nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 2, stride=2), nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# -----------------------------
# 4. Data Preparation
# -----------------------------
dataset = CombinedLaneDataset(extract_path)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

# -----------------------------
# 5. Model Setup
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LaneCNN().to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# -----------------------------
# 6. Training Loop
# -----------------------------
for epoch in range(30):  # Increase to 20 epochs
    model.train()
    epoch_loss = 0

    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

# -----------------------------
# 7. IoU Accuracy Calculation
# -----------------------------
def compute_iou(preds, targets, threshold=0.5):
    preds = preds > threshold
    targets = targets > 0.5
    intersection = (preds & targets).sum().item()
    union = (preds | targets).sum().item()
    return intersection / union if union != 0 else 1.0

total_iou = 0
model.eval()

with torch.no_grad():
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        preds = model(inputs)
        total_iou += compute_iou(preds.cpu() > 0.5, targets.cpu() > 0.5)

print(f"Mean IoU Accuracy: {total_iou / len(train_loader):.4f}")

# -----------------------------
# 8. Visualization with Noise Removal
# -----------------------------
model.eval()
inputs, targets = next(iter(train_loader))
inputs = inputs.to(device)

with torch.no_grad():
    preds = model(inputs).cpu()

for i in range(len(inputs)):
    input_img = np.transpose(inputs[i].cpu().numpy(), (1, 2, 0)) * 0.5 + 0.5  # Unnormalize to 0-1
    target_img = targets[i].cpu().squeeze().numpy()
    pred_img = preds[i].squeeze().numpy()

    # Threshold prediction
    pred_mask = (pred_img > 0.5).astype(np.uint8) * 255

    # Morphological operations to reduce noise
    kernel = np.ones((5, 5), np.uint8)
    cleaned_mask = cv2.morphologyEx(pred_mask, cv2.MORPH_OPEN, kernel)
    cleaned_mask = cv2.morphologyEx(cleaned_mask, cv2.MORPH_CLOSE, kernel)

    # Smooth sharp edges (optional)
    cleaned_mask = cv2.GaussianBlur(cleaned_mask, (3, 3), 0)

    # Normalize for display
    cleaned_mask = cleaned_mask / 255.0

    fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    axs[0].imshow(input_img)
    axs[0].set_title("Input Frame")
    axs[1].imshow(target_img, cmap='gray')
    axs[1].set_title("Ground Truth")
    axs[2].imshow(cleaned_mask, cmap='gray')
    axs[2].set_title("Predicted Lane (Cleaned)")

    plt.show()


✅ Unzipped /content/combined_dataset.zip to /content/combined_dataset
Epoch 1, Loss: 77.4182
Epoch 2, Loss: 73.5788
Epoch 3, Loss: 72.4825
Epoch 4, Loss: 71.6928
Epoch 5, Loss: 71.3041
Epoch 6, Loss: 71.0284
Epoch 7, Loss: 70.8140
Epoch 8, Loss: 70.6379
Epoch 9, Loss: 70.5051
Epoch 10, Loss: 70.3897
Epoch 11, Loss: 70.3002
Epoch 12, Loss: 70.1524
Epoch 13, Loss: 70.1709
Epoch 14, Loss: 70.0457
Epoch 15, Loss: 69.9591
Epoch 16, Loss: 69.9196
Epoch 17, Loss: 69.8221
Epoch 18, Loss: 69.7839
Epoch 19, Loss: 69.7501
Epoch 20, Loss: 69.6787
Epoch 21, Loss: 69.5966
Epoch 22, Loss: 69.5763
Epoch 23, Loss: 69.6031
Epoch 24, Loss: 69.5758
Epoch 25, Loss: 69.5187
Epoch 26, Loss: 69.4521
Epoch 27, Loss: 69.3790
